In [16]:
import numpy as np

sequence = []
chunk = np.array([0,1,2,3])
for i in range(50):
    sequence = np.append(sequence, chunk)

np.random.shuffle(sequence)
count = 0
for i in range(len(sequence)):
    while sequence[i] == sequence [i -1]:
        a = sequence[:i]
        b = sequence[i:]
        np.random.shuffle(b)
        sequence = np.append(a,b)  
print(sequence)

[ 0.  1.  3.  2.  0.  3.  1.  2.  3.  2.  3.  0.  3.  0.  1.  0.  1.  0.
  2.  0.  1.  0.  2.  3.  2.  0.  2.  0.  1.  3.  0.  3.  1.  2.  1.  2.
  1.  0.  1.  2.  0.  3.  2.  1.  3.  1.  0.  2.  3.  2.  3.  0.  3.  0.
  2.  3.  2.  3.  0.  3.  2.  1.  2.  0.  2.  3.  1.  3.  1.  3.  0.  1.
  0.  2.  0.  3.  0.  1.  0.  1.  3.  1.  2.  3.  1.  2.  1.  0.  2.  0.
  2.  1.  0.  1.  0.  2.  0.  1.  0.  2.  1.  3.  2.  3.  1.  0.  2.  0.
  3.  0.  1.  0.  1.  3.  1.  3.  0.  3.  1.  3.  1.  3.  1.  3.  0.  2.
  0.  2.  0.  3.  2.  1.  2.  0.  3.  2.  3.  2.  3.  2.  0.  3.  2.  1.
  2.  1.  3.  1.  2.  1.  2.  3.  2.  3.  2.  1.  2.  3.  0.  3.  0.  2.
  3.  0.  3.  1.  3.  0.  3.  1.  3.  1.  0.  3.  1.  0.  2.  1.  2.  1.
  3.  0.  3.  1.  2.  0.  2.  1.  2.  0.  1.  2.  1.  2.  0.  1.  3.  0.
  2.  1.]


In [21]:
import numpy as np
import random as rm

'''
To use this class:
1.) Create a seqGenerator object with optional argument int "num_states"
2.) Use makeBlockDeterministic or makeBlockProbabilistic
    to get a sequence of length equal to number of possible
    state transitions
'''

class seqGenerator():
    def __init__(self, num_states=4):
        #desired_length = 12
        # note - desired length is hard coded in makeBlock
        self.sequence = []
        
        self.num_states = num_states 
        
        # instead of crashing or throwing a warning, we keep track of
        # how many incorrect transitions there are
        self.error_counter = 0
    
        #initialize the matrix, all diagonals = -1
        self.adj_matrix = np.zeros((self.num_states,self.num_states))
        for i in range(self.num_states):
            for j in range(self.num_states):
                self.adj_matrix[i][j] = 0
   
    # this function finds the next state by attempting to greedily choose
    # in order to keep the number of transitions 'even'.
    def __findNextDeterministic(self, current):  
        #max is updated as we search for the next state with the max number
        # of zero entries in its AM row
        max = 0
        
        for j in range(self.num_states):
            # if the column is not the same state as current state
            if current != j:
                # if this transition has not already occured
                if self.adj_matrix[current][j] == 0:
                    # see how many null transitions the canditate next state has
                    count = np.sum(1-self.adj_matrix[j][:])
                    if  count > max:
                        max = count
                        next = j                 
                        
        assert type(next) == int, "next not found"
        return next

     # this function finds the next state by attempting to greedily choose
    # in order to keep the number of transitions 'even'.
    def __findNextProbabilistic(self, current):  
        #max is updated as we search for the next state with the max number
        # of zero entries in its AM row
        max = 0
        next_candidate = np.zeros(self.num_states, dtype = int)
        next_candidate.fill(-1)
        for j in range(self.num_states):
            # if the column is not the same state as current state
            if current != j:
                # if this transition has not already occured
                if self.adj_matrix[current][j] == 0:
                    # see how many null transitions the canditate next state has
                    count = np.sum(1-self.adj_matrix[j][:])
                    if  count > max:
                        #reset candidates
                        next_candidate.fill(-1)
                        next_candidate[0] = j
                        max = count
                        next = j
                    elif count == max:
                        # find next non negative entry in next_candidate
                        i = np.argmax(next_candidate >= 0)
                        next_candidate[i] = j
        
        i = np.argmax(next_candidate >= 0)
        pick = rm.randint(0,i)
        next = next_candidate[pick]
        return int(next)
    
    
    def makeBlockDeterministic(self, state):
        self.sequence.append(state)
    
        # if not first element in sequence, mark the transition, 
        # and track sum of each row (transitions)
        if len(self.sequence) != 1:
            prev = self.sequence[len(self.sequence) - 2]    #this should be the value of prev state
            self.adj_matrix[prev][state] += 1       #we increment to mark that this transition has occured
       
        next_state = self.__findNextDeterministic(state)
    
        # if we reached our desired length, append the next state and return
        if(len(self.sequence) == 12):
            return self.sequence.append(next_state)
    
        #else keep building
        return self.makeBlockDeterministic(next_state)
    
    def makeBlockProbabilistic(self, state):
        self.sequence.append(state)
    
        # if not first element in sequence, mark the transition, 
        # and track sum of each row (transitions)
        if len(self.sequence) != 1:
            prev = self.sequence[len(self.sequence) - 2]    #this should be the value of prev state
            self.adj_matrix[prev][state] += 1       #we increment to mark that this transition has occured
       
        next_state = self.__findNextProbabilistic(state)
    
        # if we reached our desired length, append the next state and return
        if(len(self.sequence) == 12):
            return self.sequence.append(next_state)
    
        #else keep building
        return self.makeBlockProbabilistic(next_state)
 

    ####################################################################
    # Use these to print sequence, counts, transitions
    ####################################################################

    def getCounts(self):
        unique, counts = np.unique(self.sequence, return_counts=True)
        print(dict(zip(unique, counts)))
        
    def getSequence(self):
        print(self.sequence)
        
    def getTransitions(self):
        print(self.adj_matrix)
        

In [22]:
block = seqGenerator(4)
seq = block.makeBlockProbabilistic(1)
block.getSequence()
block.getTransitions()

[1, 3, 2, 0, 3, 1, 2, 3, 0, 2, 1, 0, 1]
[[ 0.  0.  1.  1.]
 [ 1.  0.  1.  1.]
 [ 1.  1.  0.  1.]
 [ 1.  1.  1.  0.]]
